In [ ]:
!pip install feature_engine 2>/dev/null 1>&2
!pip install fastparquet 2>/dev/null 1>&2

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random

from sklearn.preprocessing import StandardScaler
from feature_engine.wrappers import SklearnTransformerWrapper as SKWrapper
from sklearn.model_selection import train_test_split


from matplotlib import pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Downloading data**

In [ ]:
INPUT = '../input/tabular-playground-series-oct-2022/'

df_train_dtypes = pd.read_csv(INPUT + 'train_dtypes.csv')
df_test_dtypes = pd.read_csv(INPUT + 'test_dtypes.csv')
train_dtypes = {k: v for (k, v) in zip(df_train_dtypes.column, df_train_dtypes.dtype)}
test_dtypes = {k: v for (k, v) in zip(df_test_dtypes.column, df_test_dtypes.dtype)}

train_list = []
num = 1
for i in range(num):
#     df = pd.read_csv(INPUT + f'train_{i}.csv', dtype = train_dtypes)
    df = pd.read_csv(INPUT + f'train_{i}.csv')
    df.to_parquet(f'train_{i}.parquet.gzip', compression='gzip')
    print('Done with File', i)
    train_list.append(pd.read_parquet(f'train_{i}.parquet.gzip'))

# dft = pd.read_csv(INPUT + 'test.csv', dtype = test_dtypes)
dft = pd.read_csv(INPUT + 'test.csv')
dft.to_parquet('test.parquet.gzip', compression='gzip')
print('Done with File test')
df_test = pd.read_parquet('test.parquet.gzip')
df_sample = pd.read_csv(INPUT + 'sample_submission.csv')

**Preprocessing data**

Thanks to @Jose Cáliz for feature engineering ideas!

In [ ]:
# for i in range(num):
#     print(train_list[i].shape)
#     games = random.sample(list(train_list[i].game_num.unique()), 150)
#     train_list[i] = train_list[i][train_list[i].game_num.isin(games)]
#     print(train_list[i].shape)

In [ ]:
for i in range(num):
    train_list[i]['label'] = train_list[i].team_A_scoring_within_10sec + train_list[i].team_B_scoring_within_10sec.replace(1, 2)
    train_list[i].label.value_counts(True).to_frame(name='label proportion')

In [ ]:
for i in range(num):
    train_list[i]['ball_distance_to_goal_A'] = np.sqrt(
        (train_list[i].ball_pos_x)**2 + (train_list[i].ball_pos_y + 100)**2
    )
    train_list[i]['ball_distance_to_goal_B'] = np.sqrt(
        (train_list[i].ball_pos_x)**2 + (train_list[i].ball_pos_y - 100)**2
    )
    
df_test['ball_distance_to_goal_A'] = np.sqrt(
        (df_test.ball_pos_x)**2 + (df_test.ball_pos_y + 100)**2
    )
df_test['ball_distance_to_goal_B'] = np.sqrt(
        (df_test.ball_pos_x)**2 + (df_test.ball_pos_y - 100)**2
    )

In [ ]:
columns_to_keep = [
    'ball_pos_y', 'ball_pos_x', 'ball_vel_y',
    'p0_pos_y', 'p1_pos_y', 'p2_pos_y',
    'p3_pos_y', 'p4_pos_y', 'p5_pos_y', 
    'ball_distance_to_goal_A', 'ball_distance_to_goal_B'
]
target = []
train = []
for i in range(num):
    target.append(pd.get_dummies(train_list[i]['label']))
#     target.append(train_list[i][['team_A_scoring_within_10sec','team_B_scoring_within_10sec']])
    train.append(train_list[i].drop(['game_num', 'event_id', 'event_time', 'player_scoring_next', 'team_scoring_next', 'team_A_scoring_within_10sec', 'team_B_scoring_within_10sec', 'label'], axis = 1))
#     train.append(train_list[i][columns_to_keep])

for i in range(num):
    target[i].columns = ['nobody_scores', 'team_A_scores', 'team_b_scores']
    
test = df_test.drop(['id'], axis = 1)
# test = df_test[columns_to_keep]

In [ ]:
for i in range(num):
    train[i] = train[i].fillna(train[i].median())
#     train[i] = train[i].fillna(0)
    
test = test.fillna(test.median())
# test = test.fillna(0)

In [ ]:
# scaler = StandardScaler()
scaler = SKWrapper(StandardScaler(), variables=train[0].columns.tolist())
scaler.fit(train[0])
for i in range(num):
    train[i] = scaler.transform(train[i])
test = scaler.transform(test)

In [ ]:
# X_train, X_valid, y_train, y_valid = train_test_split(train[0], target[0], test_size = 0.2, shuffle=True)

**Model evaluation**

In [ ]:
import lightgbm as lgb

train_data_A = []
train_data_B = []
for i in range(num):
    train_data_A.append(lgb.Dataset(train[i], label=target[i]['team_A_scores'], free_raw_data=False))
    train_data_B.append(lgb.Dataset(train[i], label=target[i]['team_b_scores'], free_raw_data=False))

params = {}

gbm_A = []
gbm_A.append(lgb.train(params, train_data_A[0], num_boost_round = 50))
for i in range(1, num):
    gbm_A.append(lgb.train(params, train_data_A[i], num_boost_round = 50, init_model = gbm_A[i-1]))

predictionA = gbm_A[num-1].predict(test)

gbm_B = []
gbm_B.append(lgb.train(params, train_data_B[0], num_boost_round = 50))
for i in range(1, num):
    gbm_B.append(lgb.train(params, train_data_B[i], num_boost_round = 50, init_model = gbm_B[i-1]))

predictionB = gbm_B[num-1].predict(test)


# gbm_initA = lgb.train(params, train_data_A0, num_boost_round = 50)
# print("Initial iter# %d" %gbm_initA.current_iteration())
# gbm_2A = lgb.train(params, train_data_A1, num_boost_round = 50,
#                 init_model = gbm_initA)
# print("Option 2 current iter# %d" %gbm_2A.current_iteration())
# predictionA = gbm_2A.predict(test)


# gbm_initB = lgb.train(params, train_data_B0, num_boost_round = 50)
# print("Initial iter# %d" %gbm_initB.current_iteration())
# gbm_2B = lgb.train(params, train_data_B1, num_boost_round = 50,
#                 init_model = gbm_initB)
# print("Option 2 current iter# %d" %gbm_2B.current_iteration())
# predictionB = gbm_2B.predict(test)



# gbmA = lgb.LGBMRegressor(boosting_type='gbdt',
#                         num_leaves=50,
#                         learning_rate=0.01,
#                         max_depth=-1,
#                         n_estimators=1000)

# gbmA.fit(train[0], target[0]['team_A_scores'],
#         eval_set=[(train[1], target[1]['team_A_scores'])],
#         eval_metric='l1',
#         callbacks=[lgb.early_stopping(5)])

# predictionA = gbmA.predict(test, num_iteration=gbmA.best_iteration_)

# gbmB = lgb.LGBMRegressor(boosting_type='gbdt',
#                         num_leaves=50,
#                         learning_rate=0.01,
#                         max_depth=-1,
#                         n_estimators=1000)

# gbmB.fit(train[0], target[0]['team_b_scores'],
#         eval_set=[(train[1], target[1]['team_b_scores'])],
#         eval_metric='l1',
#         callbacks=[lgb.early_stopping(5)])

# predictionB = gbmB.predict(test, num_iteration=gbmB.best_iteration_)

In [ ]:
# gbmA.feature_importances_

In [ ]:
predictionA

**Making prediction**

In [ ]:
preds = pd.DataFrame([predictionA, predictionB], 
#                     columns=['team_A_scoring_within_10sec', 'team_B_scoring_within_10sec'],
#                     index=test.index
                    )
# preds = np.round(preds).astype(int)

In [ ]:
preds = preds.T
preds.columns = ['team_A_scoring_within_10sec', 'team_B_scoring_within_10sec']
preds

In [ ]:
df_sample[['team_A_scoring_within_10sec', 'team_B_scoring_within_10sec']] = preds[['team_A_scoring_within_10sec', 'team_B_scoring_within_10sec']]

In [ ]:
df_sample.to_csv('submission.csv', index = False)